# Pathology prediction (EDEMA)


@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the EDEMA pathology using the CSV embeddings file


## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples : 0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value.

The model is based on binary classification for each target chest pathology.

Final sample size for the EDEMA pathology is : (N = 17,182)

#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.EDEMA,
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Edema", experiment_name="CP_Edema", filepath="./results/edema")

# Model training and results evaluation
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='stratifiedkfold',
    outer_fold=5,
    outer_strategy='stratifiedkfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-09-24 16:07:16,490	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=121391) Outer fold 1


(raylet) Spilled 2750 MiB, 3 objects, write throughput 586 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
Processing:   0%|          | 0/4 [00:00<?, ?it/s]


(run_fold pid=121391) Configuring PyCaret for outer fold 1
(run_fold pid=121386) Outer fold 2


Processing:  75%|███████▌  | 3/4 [07:56<02:56, 176.16s/it] [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
                                                          
Processing:  75%|███████▌  | 3/4 [07:55<02:56, 176.09s/it]


...

(run_fold pid=121386) Configuring PyCaret for outer fold 5


Processing:  25%|██▌       | 1/4 [00:01<00:03,  1.29s/it]


(run_fold pid=121391) Transformation Pipeline and Model Successfully Saved
(run_fold pid=121391)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=121391) 0  Extreme Gradient Boosting    0.8577  0.9257  ...  0.8894  0.6904  0.6932
(run_fold pid=121391) 
(run_fold pid=121391) [1 rows x 8 columns]


Processing:  75%|███████▌  | 3/4 [07:34<02:48, 168.37s/it]
                                                          


(run_fold pid=121386)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=121386) Fold                                                          
(run_fold pid=121386) 0       0.8350  0.9053  0.8854  0.8548  0.8698  0.6447  0.6455
(run_fold pid=121386) 1       0.8386  0.9076  0.9028  0.8477  0.8744  0.6491  0.6516
(run_fold pid=121386) 2       0.8477  0.9090  0.9014  0.8605  0.8805  0.6708  0.6721
(run_fold pid=121386) 3       0.8427  0.9037  0.9000  0.8551  0.8770  0.6592  0.6609
(run_fold pid=121386) 4       0.8404  0.9038  0.9066  0.8478  0.8762  0.6523  0.6552
(run_fold pid=121386) Mean    0.8409  0.9059  0.8992  0.8532  0.8756  0.6552  0.6571
(run_fold pid=121386) Std     0.0042  0.0021  0.0073  0.0049  0.0035  0.0091  0.0091
(run_fold pid=121386) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=121386) Transformation Pipeline and Model Successfully Saved
(run_fold pid=121386)                        Model  Accuracy  